In [1]:
import json
import os
import re


archivo_json = "../threatFox/threatFox.json"
salida_json = "../threatFox/threatFox_con_sha256.json"
data_con_sha256 = []

def is_sha256(iocs):
    patron_sha256 = re.compile(r'^[a-fA-F0-9]{64}$')
    return bool(patron_sha256.match(iocs))

if os.path.exists(archivo_json):
    with open(archivo_json, "r") as file:
        data_json = json.load(file)

        for item in data_json:
            if "ioc_type" in item and item["ioc_type"] == "sha256_hash":
                if "ioc" in item:
                    if is_sha256(item["ioc"]):
                        print(f"SHA256: {item['ioc']}")
                        # Crea un diccionario para cada objeto con sha256
                        sha256_info = {
                            "desc": item.get("threat_type_desc", ""),
                            "iocs": item.get("ioc", ""),
                            "ioctype": "",  # Debes ajustar este campo según tu lógica
                            "SHA256": item.get("ioc", ""),  # Recoge el hash
                            "threat_type": item.get("threat_type", ""),  # Recoge el tipo de amenaza
                            "malware": item.get("malware", ""),  # Nombre del malware
                            "malware_alias": "",  # Puedes ajustar este campo según tu lógica
                            "family": ""  # Puedes ajustar este campo según tu lógica
                        }
                        data_con_sha256.append(sha256_info)

    # Crea la estructura final
    json_result = {
        "malwares": data_con_sha256,
    }

    with open(salida_json, "w") as archivo_salida:
        json.dump(json_result, archivo_salida, indent=4)
        print(f"Datos escritos en {salida_json}")


SHA256: 87122e2d4341a44298b65c15f36571b1a9aba0b89717acd5cd3f8368cde034ab
SHA256: 68e494c702789bcab12260ded73112ff54f2228a9865f57f20b901d9ba0561f3
SHA256: 318799146F72F9938BF3873F06072FD923F980E05E12264D3392719AD9C37AE2
SHA256: EFFA6FCB8759375B4089CCF61202A5C63243F4102872E64E3EB0A1BDC2727659
SHA256: E604B855F5A7C411D7EAFC0079EFE2DF6BC30FD066CF0C7C80FDC32398B16EB2
SHA256: 9A43C57F3E98BD69789E8CCBEEF2C1B6B5A3B1D06D63257BB4BD58DFFA23689D
SHA256: 6E908F8726ECA4021343C033F91E35330947D0DD82DE43CEE1304F267AC8B2E8
Datos escritos en ../threatFox/threatFox_con_sha256.json


In [84]:
import requests
import json
import re

id_triage_list = []
archivo_json = "../threatFox/threatFox_con_sha256.json"

with open(archivo_json, "r") as file:
    threat_fox_data = json.load(file)
    malwares_list = threat_fox_data.get("malwares", [])

# API key
key = "2a6dadcca0fdcf1ac794701345bbbb71480bcfcf"

def es_sha256(cadena):
    patron_sha256 = re.compile(r'^[a-fA-F0-9]{64}$')
    return bool(patron_sha256.match(cadena))

# Lista para almacenar resultados
resultados = []

for malware in malwares_list:
    sha256_malware = malware.get("SHA256", "")
    malware_name = malware.get("malware", "")
    threat_type = malware.get("threat_type", "")

    if es_sha256(sha256_malware):
        url = f"https://tria.ge/api/v0/search?query=sha256:{sha256_malware}"
        response = requests.get(url, headers={"Authorization": "Bearer " + key})

        if response.status_code == 200:
            samples = response.json().get("data", [])
            if samples:
                id_triage = samples[0].get("id", "")
            else:
                print("No se encontraron muestras para el SHA256:", sha256_malware)
                continue

            url = f"https://tria.ge/api/v0/samples/{id_triage}/overview.json"
            response = requests.get(url, headers={"Authorization": "Bearer " + key})

            if response.status_code == 200:
                result_json = json.dumps(response.json(), indent=4, sort_keys=True)
                analysis_data = response.json().get("analysis")
                extracted_data = response.json().get("extracted")
                sample_data = response.json().get("sample")
                signatures_list = response.json().get("signatures", [])

                sha256_muestra = sample_data.get("sha256", "")

                ttp_data = set()
                tactic_list = set()

                for signa in signatures_list:
                    ttp_values = signa.get("ttp", [])
                    for ttp_value in ttp_values:
                        if ttp_value != "":
                            if "." in ttp_value:
                                tactic_list.add(ttp_value)
                            else:
                                ttp_data.add(ttp_value)

                desc = next((signature.get("desc", "") for signature in signatures_list if signature.get("desc")), "")

                malware_aliases = set()
                malware_alias_list = analysis_data.get("malware_alias", [])
                for alias in malware_alias_list:
                    malware_aliases.add(alias)

                iocs_list = response.json().get("targets", [])
                iocs = [target.get("iocs", []) for target in iocs_list]

                malware_json = {
                    "details": {
                        "desc": desc,
                        "iocs": iocs,
                        "ioctype": analysis_data.get("ioctype", ""),
                        "SHA256": sha256_muestra,
                        "threat_type": threat_type,
                        "malware": malware_name,
                        "malware_alias": list(malware_aliases),
                        "family": analysis_data.get("family", ""),
                    },
                    "vulnerability": {},
                    "techniques": list(ttp_data),
                    "tactic": list(tactic_list),
                    "tool": {},
                    "threat_actor": [],
                    "asset": {},
                    "relationships": {},
                }
                resultados.append(malware_json)
            else:
                print("Error al obtener datos para el SHA256:", sha256_malware)
        else:
            print("Error al realizar la búsqueda para el SHA256:", sha256_malware)

# Guardar la lista de resultados como un archivo JSON
with open("malwares-test.json", "w") as file:
    json.dump(resultados, file, indent=4)


In [13]:
import requests
from bs4 import BeautifulSoup
import json
import os
        
def search_apt(malware_value):
    html = requests.get('https://docs.google.com/spreadsheets/u/1/d/1H9_xaxQHpWaa4O_Son4Gx0YOIzlcBWMsdvePFX68EKU/pubhtml#').text

    soup = BeautifulSoup(html, 'html.parser')
    tables = soup.find_all('td')
    actors = []
    # Busca el índice de la columna "Toolset / Malware"
    target_column = None
    for i, tool in enumerate(tables):
        header = tool.find_parent('th')
        if "Toolset / Malware" in tool.text.strip():
            target_column = i
            break

    # Verificar si se encontró la columna
    if target_column is not None:
        # Iterar sobre las filas y extraer el texto de la columna con el índice encontrado
        for i in range(target_column, len(tables) - target_column):
            text_toolset = tables[i].text.strip().upper()
            if (str(malware_value)).upper() in text_toolset:
                target_tr = tables[i].find_parent('tr')
                print(target_tr)
                if target_tr:
                    tds_in_row = target_tr.find_all('td')
                    if tds_in_row:
                        first_td_in_row = tds_in_row[0]
                    print(first_td_in_row)  # Primer td de la fila
                    actors.append(first_td_in_row.text.strip())
                    print(actors)
        return actors
    else:
        print("No se encontró la columna 'Toolset / Malware'")
data = '{"source":"threat_intel","asset":{},"message":{"malware_alias":[],"malware":"win.poison","iocs":[{"urls":["https://g.bing.com/neg/0?action=emptycreativeimpression&adUnitId=11730597&publisherId=251978541&rid=c49daf0978374fadaedc3e4010ecf8e3&localId=w:09BE301F-AF04-8909-0529-ADD134954281&deviceId=6896190259413427&anid=","https://g.bing.com/neg/0?action=emptycreative&adUnitId=11730597&publisherId=251978541&rid=c49daf0978374fadaedc3e4010ecf8e3&localId=w:09BE301F-AF04-8909-0529-ADD134954281&deviceId=6896190259413427&anid="],"domains":["2.tcp.eu.ngrok.io","85.177.190.20.in-addr.arpa","240.221.184.93.in-addr.arpa","9.228.82.20.in-addr.arpa","g.bing.com","200.197.79.204.in-addr.arpa","41.110.16.96.in-addr.arpa","55.36.223.20.in-addr.arpa","95.221.229.192.in-addr.arpa","57.138.127.3.in-addr.arpa","86.23.85.13.in-addr.arpa","198.187.3.20.in-addr.arpa","18.134.221.88.in-addr.arpa","23.236.111.52.in-addr.arpa","178.223.142.52.in-addr.arpa","180.178.17.96.in-addr.arpa","105.193.132.51.in-addr.arpa"],"ips":["8.8.8.8","3.127.138.57","18.197.239.5","18.156.13.209","138.91.171.81","204.79.197.200"]}],"vulnerability":"{}","tool":"{}","tactic":["T1543.003"],"SHA256":"c93ab6bb562f09706d141a4804e655fe92612a07bc3ab92bf1f6f7a7a9ef9dcc","relationships":"{}","threat_actor":"[]","ioctype":"","techniques":["T1102","T1012","T1082"],"threat_type":"botnet_cc","family":["njrat"],"asset":"1222664","desc":"Widely used RAT written in .NET."}}'
data = json.loads(data)
malware_value = data.get('message', {}).get('malware')
malware_value = malware_value.split('.')[1].upper()
print(malware_value)
if malware_value:
    #print("Buscando en APT Notes el malware:", malware_value)
    data['message']['threat_actor'] = search_apt(malware_value)
directory = 'enriquecidos'
if not os.path.exists(directory):
    os.makedirs(directory)
with open(os.path.join(directory, 'malware-apt-test.json'), 'w') as outfile:
    json.dump(data, outfile, indent=4)
print(data)
print("Datos enriquecidos guardados en enriquecidos/malware-apt-test.json")



POISON
<tr style="height: 20px"><th class="row-headers-background row-header-shim" id="361554658R6" style="height: 20px;"><div class="row-header-wrapper" style="line-height: 20px">7</div></th><td class="s38" dir="ltr">APT16</td><td class="freezebar-cell"></td><td class="s6"></td><td class="s6"></td><td class="s6"></td><td class="s8"></td><td class="s38" dir="ltr">APT16</td><td class="s6"></td><td class="s6"></td><td class="s6"></td><td class="s6"></td><td class="s6"></td><td class="s8"></td><td class="s34" dir="ltr"><a href="https://www.google.com/url?q=https://attack.mitre.org/groups/G0023/&amp;sa=D&amp;source=editors&amp;ust=1703156440445991&amp;usg=AOvVaw0O8gSzs_RuxsbzG6jopstM" rel="noreferrer" target="_blank">G0023</a></td><td class="s39"></td><td class="s6"></td><td class="s6"></td><td class="s6"></td><td class="s8" dir="ltr">ELMER backdoor, Gh0st, HTRAN, UNICAT,Poison Ivy, Pandora</td><td class="s8" dir="ltr">This threat actor targets and compromises Japanese and Taiwanese entiti

### SCALA

In [26]:
import pandas as pd
import json
import requests
url_download = "https://docs.google.com/spreadsheets/d/1H9_xaxQHpWaa4O_Son4Gx0YOIzlcBWMsdvePFX68EKU/pub?output=xlsx"
response = requests.get(url_download)
if response.status_code == 200:
    with open("apt_notes.xlsx", "wb") as file:
        file.write(response.content)
        print("Archivo descargado correctamente")
else:
    print("Error al descargar el archivo")

Archivo descargado correctamente


In [42]:
import pandas as pd
countries = {'China':17,'Russia':22,'North Korea':23,'Iran':15,'Israel':7,'NATO':12,'Middle East':9,'Others':12}
for k,v in countries.items():
    df = pd.read_excel('apt_notes.xlsx',sheet_name=k,engine='openpyxl')
    c = k
    if (k == 'Others'):
        c = "Other Actors"
    CommonNames = df[c].tolist()
    del CommonNames[0]
    tools_column = "Unnamed: " + str(v)
    tools = df[tools_column].tolist()
    del tools[0]
    result = {CommonNames[i]: tools[i] for i in range(len(CommonNames))}
    #Change Nan for null
    result = {k: None if pd.isnull(v) else v for k, v in result.items()}
    countries[k] = result

print(countries)

{'China': {'Comment Crew': 'WEBC2, BISCUIT and many others', 'APT2': 'Their activities are commonly known to be exploiting CVE-2012-0158 (MSOffice vulnerability in MSCOMCTL.OCX) in SpearPhising operations. Related malware: Moose, Warp, MSUpdater', 'UPS': 'Shotput, Pirpi, PlugX/Sogu, Kaba, Cookie Cutter, many 0days: IE, Firefox, and Flash, SportLoader, Shadow Brokers exploits, DoublePulsar, Bemstour, Filensfer', 'IXESHE': 'Etumbot, Riptide, Hightide, ThreeByte, Waterspout, Mswab, Gh0st, ShowNews, 3001', 'APT16': 'ELMER backdoor, Gh0st, HTRAN, UNICAT,Poison Ivy, Pandora', 'Hidden Lynx': 'BLACKCOFFEE, WEBCnC, Joy RAT, PlugX, Trojan.Naid, Backdoor.Moudoor, Backdoor.Vasport, Backdoor.Boda, Trojan.Hydraq, ZxShell, Sakula, China Chopper, DestroyRAT', 'Wekby': 'HTTPBrowser, TokenControl, HcdLoader, PisLoader, TManger', 'Axiom': 'Winnti, Gh0st RAT, PoisonIvy, HydraQ, Hikit, ZxShell, Deputy Dog, Derusbi, PlugX, HTRAN, HDRoot, Fscan, Timestomper', 'Winnti Group': 'Winnti, AceHash, PlugX, Webshell

In [51]:
#Iterate countries for search the actor using tools
tool = "Poison"
tool = tool.upper().strip()
apts = []
for k in countries:
    for k2,v2 in countries[k].items():
        if v2 is not None:
            if tool in v2.upper().strip():
                apts.append(k2)

In [53]:
apts

['APT16',
 'Axiom',
 'APT6',
 'Anchor Panda',
 'Nightshade Panda',
 'Stone Panda',
 'Temper Panda',
 'Violin Panda',
 'Tropic Trooper',
 'APT41',
 'APT-C-01',
 'GALLIUM',
 'APT5',
 'Tortoiseshell',
 'Molerats']

In [54]:
data = '{"source":"threat_intel","asset":{},"message":{"malware_alias":[],"malware":"win.poison","iocs":[{"urls":["https://g.bing.com/neg/0?action=emptycreativeimpression&adUnitId=11730597&publisherId=251978541&rid=c49daf0978374fadaedc3e4010ecf8e3&localId=w:09BE301F-AF04-8909-0529-ADD134954281&deviceId=6896190259413427&anid=","https://g.bing.com/neg/0?action=emptycreative&adUnitId=11730597&publisherId=251978541&rid=c49daf0978374fadaedc3e4010ecf8e3&localId=w:09BE301F-AF04-8909-0529-ADD134954281&deviceId=6896190259413427&anid="],"domains":["2.tcp.eu.ngrok.io","85.177.190.20.in-addr.arpa","240.221.184.93.in-addr.arpa","9.228.82.20.in-addr.arpa","g.bing.com","200.197.79.204.in-addr.arpa","41.110.16.96.in-addr.arpa","55.36.223.20.in-addr.arpa","95.221.229.192.in-addr.arpa","57.138.127.3.in-addr.arpa","86.23.85.13.in-addr.arpa","198.187.3.20.in-addr.arpa","18.134.221.88.in-addr.arpa","23.236.111.52.in-addr.arpa","178.223.142.52.in-addr.arpa","180.178.17.96.in-addr.arpa","105.193.132.51.in-addr.arpa"],"ips":["8.8.8.8","3.127.138.57","18.197.239.5","18.156.13.209","138.91.171.81","204.79.197.200"]}],"vulnerability":"{}","tool":"{}","tactic":["T1543.003"],"SHA256":"c93ab6bb562f09706d141a4804e655fe92612a07bc3ab92bf1f6f7a7a9ef9dcc","relationships":"{}","threat_actor":"[]","ioctype":"","techniques":["T1102","T1012","T1082"],"threat_type":"botnet_cc","family":["njrat"],"asset":"1222664","desc":"Widely used RAT written in .NET."}}'
data = json.loads(data)
malware_value = data.get('message', {}).get('malware')
malware_value = malware_value.split('.')[1].upper()
apts = []
for k in countries:
    for k2,v2 in countries[k].items():
        if v2 is not None:
            if malware_value in v2.upper().strip():
                apts.append(k2)
data["message"]["threat_actor"] = apts
print(data)

{'source': 'threat_intel', 'asset': {}, 'message': {'malware_alias': [], 'malware': 'win.poison', 'iocs': [{'urls': ['https://g.bing.com/neg/0?action=emptycreativeimpression&adUnitId=11730597&publisherId=251978541&rid=c49daf0978374fadaedc3e4010ecf8e3&localId=w:09BE301F-AF04-8909-0529-ADD134954281&deviceId=6896190259413427&anid=', 'https://g.bing.com/neg/0?action=emptycreative&adUnitId=11730597&publisherId=251978541&rid=c49daf0978374fadaedc3e4010ecf8e3&localId=w:09BE301F-AF04-8909-0529-ADD134954281&deviceId=6896190259413427&anid='], 'domains': ['2.tcp.eu.ngrok.io', '85.177.190.20.in-addr.arpa', '240.221.184.93.in-addr.arpa', '9.228.82.20.in-addr.arpa', 'g.bing.com', '200.197.79.204.in-addr.arpa', '41.110.16.96.in-addr.arpa', '55.36.223.20.in-addr.arpa', '95.221.229.192.in-addr.arpa', '57.138.127.3.in-addr.arpa', '86.23.85.13.in-addr.arpa', '198.187.3.20.in-addr.arpa', '18.134.221.88.in-addr.arpa', '23.236.111.52.in-addr.arpa', '178.223.142.52.in-addr.arpa', '180.178.17.96.in-addr.arpa',

## Traduccion

In [55]:
import pandas as pd
import json
import requests
url_download = "https://docs.google.com/spreadsheets/d/1H9_xaxQHpWaa4O_Son4Gx0YOIzlcBWMsdvePFX68EKU/pub?output=xlsx"
response = requests.get(url_download)
if response.status_code == 200:
    with open("apt_notes.xlsx", "wb") as file:
        file.write(response.content)
        print("Archivo descargado correctamente")
else:
    print("Error al descargar el archivo")
countries = {'China':17,'Russia':22,'North Korea':23,'Iran':15,'Israel':7,'NATO':12,'Middle East':9,'Others':12}
for k,v in countries.items():
    df = pd.read_excel('apt_notes.xlsx',sheet_name=k,engine='openpyxl')
    c = k
    if (k == 'Others'):
        c = "Other Actors"
    CommonNames = df[c].tolist()
    del CommonNames[0]
    tools_column = "Unnamed: " + str(v)
    tools = df[tools_column].tolist()
    del tools[0]
    result = {CommonNames[i]: tools[i] for i in range(len(CommonNames))}
    #Change Nan for null
    result = {k: None if pd.isnull(v) else v for k, v in result.items()}
    countries[k] = result

data = '{"source":"threat_intel","asset":{},"message":{"malware_alias":[],"malware":"win.poison","iocs":[{"urls":["https://g.bing.com/neg/0?action=emptycreativeimpression&adUnitId=11730597&publisherId=251978541&rid=c49daf0978374fadaedc3e4010ecf8e3&localId=w:09BE301F-AF04-8909-0529-ADD134954281&deviceId=6896190259413427&anid=","https://g.bing.com/neg/0?action=emptycreative&adUnitId=11730597&publisherId=251978541&rid=c49daf0978374fadaedc3e4010ecf8e3&localId=w:09BE301F-AF04-8909-0529-ADD134954281&deviceId=6896190259413427&anid="],"domains":["2.tcp.eu.ngrok.io","85.177.190.20.in-addr.arpa","240.221.184.93.in-addr.arpa","9.228.82.20.in-addr.arpa","g.bing.com","200.197.79.204.in-addr.arpa","41.110.16.96.in-addr.arpa","55.36.223.20.in-addr.arpa","95.221.229.192.in-addr.arpa","57.138.127.3.in-addr.arpa","86.23.85.13.in-addr.arpa","198.187.3.20.in-addr.arpa","18.134.221.88.in-addr.arpa","23.236.111.52.in-addr.arpa","178.223.142.52.in-addr.arpa","180.178.17.96.in-addr.arpa","105.193.132.51.in-addr.arpa"],"ips":["8.8.8.8","3.127.138.57","18.197.239.5","18.156.13.209","138.91.171.81","204.79.197.200"]}],"vulnerability":"{}","tool":"{}","tactic":["T1543.003"],"SHA256":"c93ab6bb562f09706d141a4804e655fe92612a07bc3ab92bf1f6f7a7a9ef9dcc","relationships":"{}","threat_actor":"[]","ioctype":"","techniques":["T1102","T1012","T1082"],"threat_type":"botnet_cc","family":["njrat"],"asset":"1222664","desc":"Widely used RAT written in .NET."}}'
data = json.loads(data)
malware_value = data.get('message', {}).get('malware')
malware_value = malware_value.split('.')[1].upper()
apts = []
for k in countries:
    for k2,v2 in countries[k].items():
        if v2 is not None:
            if malware_value in v2.upper().strip():
                apts.append(k2)
data["message"]["threat_actor"] = apts


Archivo descargado correctamente


In [56]:
data

{'source': 'threat_intel',
 'asset': {},
 'message': {'malware_alias': [],
  'malware': 'win.poison',
  'iocs': [{'urls': ['https://g.bing.com/neg/0?action=emptycreativeimpression&adUnitId=11730597&publisherId=251978541&rid=c49daf0978374fadaedc3e4010ecf8e3&localId=w:09BE301F-AF04-8909-0529-ADD134954281&deviceId=6896190259413427&anid=',
     'https://g.bing.com/neg/0?action=emptycreative&adUnitId=11730597&publisherId=251978541&rid=c49daf0978374fadaedc3e4010ecf8e3&localId=w:09BE301F-AF04-8909-0529-ADD134954281&deviceId=6896190259413427&anid='],
    'domains': ['2.tcp.eu.ngrok.io',
     '85.177.190.20.in-addr.arpa',
     '240.221.184.93.in-addr.arpa',
     '9.228.82.20.in-addr.arpa',
     'g.bing.com',
     '200.197.79.204.in-addr.arpa',
     '41.110.16.96.in-addr.arpa',
     '55.36.223.20.in-addr.arpa',
     '95.221.229.192.in-addr.arpa',
     '57.138.127.3.in-addr.arpa',
     '86.23.85.13.in-addr.arpa',
     '198.187.3.20.in-addr.arpa',
     '18.134.221.88.in-addr.arpa',
     '23.236.11